# 02. Data Cleaning | تنظيف البيانات

## 📚 Prerequisites (What You Need First) | المتطلبات الأساسية

**BEFORE starting this notebook**, you should have completed:
- ✅ **Example 1: Data Loading and Exploration** - You need to know how to identify data quality issues first!
- ✅ **Basic pandas knowledge**: DataFrames, indexing, filtering
- ✅ **Understanding of data quality**: What are missing values, duplicates, outliers?

**If you haven't completed these**, you might struggle with:
- Understanding why data cleaning is necessary
- Knowing which cleaning method to use
- Understanding the impact of cleaning on your data

---

## 🔗 Where This Notebook Fits | مكان هذا الدفتر

**This is the SECOND example** - it fixes the problems we found in Example 1!

**Why this example SECOND?**
- **Before** you can preprocess data, you need to clean it
- **Before** you can build models, you need clean data
- **Before** you can make predictions, you need to fix data quality issues

**Builds on**: 
- 📓 Example 1: Data Loading and Exploration (we found the problems, now we fix them!)

**Leads to**: 
- 📓 Example 3: Data Preprocessing (needs clean data to work with)
- 📓 Example 4: Linear Regression (needs clean, preprocessed data)
- 📓 All ML models (all need clean data!)

**Why this order?**
1. Data cleaning fixes **quality issues** (needed before preprocessing)
2. Data cleaning teaches you **when to remove vs. impute** (critical decision-making)
3. Data cleaning shows you **the impact of outliers** (affects model accuracy)

---

## The Story: Cleaning Before Cooking | القصة: التنظيف قبل الطبخ

Imagine you're cooking a meal. **Before** you can cook, you need to clean your ingredients - remove spoiled items, wash vegetables, check for foreign objects. **After** cleaning everything, you can prepare a safe, delicious meal!

Same with machine learning: **Before** building models, we clean our data - remove duplicates, handle missing values, fix outliers. **After** cleaning, we can build accurate, reliable models!

---

## Why Data Cleaning Matters | لماذا يهم تنظيف البيانات؟

Data cleaning is essential for accurate models:
- **Missing Values**: Break ML algorithms - must be handled
- **Duplicates**: Bias your models (same data counted twice)
- **Outliers**: Skew predictions and statistics
- **Wrong Data Types**: Cause errors in calculations
- **Dirty Data = Bad Models**: No amount of ML can fix fundamentally bad data

## Learning Objectives | أهداف التعلم
1. Handle missing values (remove or impute)
2. Remove duplicate rows
3. Detect and handle outliers
4. Convert data types correctly
5. Understand trade-offs between different cleaning methods
6. Know when to remove vs. when to fix data

In [1]:
# Step 1: Import necessary libraries
# These libraries help us clean and analyze data

import pandas as pd  # For data manipulation (cleaning operations)
import numpy as np   # For numerical operations (handling NaN, calculations)
import matplotlib.pyplot as plt  # For visualizations (seeing outliers)
import seaborn as sns  # For statistical plots (data quality visualization)

print("✅ Libraries imported successfully!")
print("\n📚 What each library does:")
print("   - pandas: Clean data (remove, fill, filter)")
print("   - numpy: Handle missing values (NaN operations)")
print("   - matplotlib: Visualize data quality issues")
print("   - seaborn: Create beautiful quality check plots")

<frozen importlib._bootstrap>:219: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


✅ Libraries imported successfully!

📚 What each library does:
   - pandas: Clean data (remove, fill, filter)
   - numpy: Handle missing values (NaN operations)
   - matplotlib: Visualize data quality issues
   - seaborn: Create beautiful quality check plots


## Part 1: Setting the Scene | الجزء الأول: إعداد المشهد

**BEFORE**: We explored our data in Example 1 and found problems - missing values, duplicates, outliers.

**AFTER**: We'll clean the data by fixing all these issues, making it ready for preprocessing and modeling!

**Why this matters**: Dirty data produces unreliable models. Cleaning is non-negotiable for good ML results!

In [2]:
# Create sample data with common data quality issues
# In real projects, you'd load this from a file and discover these issues during exploration

print("\n1. Creating sample data with common issues...")
print("إنشاء بيانات نموذجية بمشاكل شائعة...")

np.random.seed(42)  # For reproducibility
data = {
    'id': range(1, 21),
    'name': [f'Person_{i}' for i in range(1, 21)],
    'age': [25, 30, None, 35, 40, None, 28, 32, 45, 50,
            22, None, 38, 42, 29, 33, 48, None, 27, 31],  # Missing values (None = NaN)
    'salary': [50000, 60000, 55000, 70000, 80000, 65000, 58000,
               72000, 90000, 95000, 52000, 68000, 75000, 85000,
               57000, 73000, 92000, 62000, 54000, 71000],
    'department': ['IT', 'HR', 'IT', 'Finance', 'IT', 'HR', 'IT',
                   'Finance', 'IT', 'HR', 'IT', None, 'Finance', 'IT',
                   'HR', 'IT', 'Finance', 'IT', None, 'HR'],  # Missing values
    'experience_years': [2, 5, 3, 8, 10, 4, 2.5, 6, 12, 15,
                         1, 5.5, 9, 11, 3, 7, 13, 4.5, 2, 6]
}
df = pd.DataFrame(data)

print("✅ Sample data created with intentional issues:")
print("   - Missing values in 'age' and 'department'")
print("   - We'll add duplicates and outliers next")



1. Creating sample data with common issues...
إنشاء بيانات نموذجية بمشاكل شائعة...
✅ Sample data created with intentional issues:
   - Missing values in 'age' and 'department'
   - We'll add duplicates and outliers next


## Step 1: Create Sample Data with Issues | الخطوة 1: إنشاء بيانات نموذجية بمشاكل

**BEFORE**: We need to learn cleaning techniques, but we need "dirty" data to practice on.

**AFTER**: We'll create sample data with common real-world problems (missing values, duplicates, outliers) so we can practice cleaning!

**Why create dirty data?** Real datasets have these problems! We need to learn how to handle them.

In [ ]:
# Add some duplicate rows to demonstrate duplicate removal
# Why add duplicates? Real data often has duplicates from data entry errors or merging issues

# pd.concat([df, df.iloc[[0, 1]]], ignore_index=True)
# - pd.concat(): Combines multiple DataFrames vertically (stacks them)
# - [df, df.iloc[[0, 1]]]: List of DataFrames to combine
#   - df: Original DataFrame
#   - df.iloc[[0, 1]]: Selects rows 0 and 1 by integer position
#     - iloc: Integer-location based indexing (selects by position, not label)
#     - [[0, 1]]: Double brackets select multiple rows, returns DataFrame
#     - This gets a copy of the first 2 rows
# - ignore_index=True: Resets index after concatenation (0, 1, 2, ... instead of 0, 1, 0, 1)
# Result: Original 20 rows + 2 duplicate rows = 22 rows total
df = pd.concat([df, df.iloc[[0, 1]]], ignore_index=True)
print("   - Added 2 duplicate rows")

   - Added 2 duplicate rows


In [ ]:
# Add some outliers to demonstrate outlier handling
# Why add outliers? Real data has outliers from errors (typos, measurement mistakes) or rare events

# df.loc[18, 'salary'] = 500000
# - df.loc[]: Label-based indexing (selects by index label, not position)
# - [18, 'salary']: Row index 18, column 'salary'
# - = 500000: Assigns new value to that cell
# - loc uses labels: if index is [0,1,2,...], loc[18] means row with label 18
df.loc[18, 'salary'] = 500000  # Extreme outlier (10x normal salary - likely a typo)

# df.loc[19, 'age'] = 150
# - Same as above, but for row 19, column 'age'
# - Sets age to 150 (impossible value for demonstration)
df.loc[19, 'age'] = 150  # Impossible value (no one lives to 150 - data entry error)

# df.shape
# - Returns tuple (rows, columns): (number_of_rows, number_of_columns)
# - Example: (22, 6) means 22 rows and 6 columns
print("\n📊 Original Data Shape:", df.shape)
print("شكل البيانات الأصلية:", df.shape)
print("   - Added 1 extreme salary outlier")
print("   - Added 1 impossible age value")

# df.head(10)
# - Returns first 10 rows of DataFrame
# - head(n): Shows first n rows (default is 5)
# - Useful for quick data inspection
print("\n📄 Original Data (first 10 rows):")
print(df.head(10))


📊 Original Data Shape: (22, 6)
شكل البيانات الأصلية: (22, 6)
   - Added 1 extreme salary outlier
   - Added 1 impossible age value

📄 Original Data (first 10 rows):
   id       name   age  salary department  experience_years
0   1   Person_1  25.0   50000         IT               2.0
1   2   Person_2  30.0   60000         HR               5.0
2   3   Person_3   NaN   55000         IT               3.0
3   4   Person_4  35.0   70000    Finance               8.0
4   5   Person_5  40.0   80000         IT              10.0
5   6   Person_6   NaN   65000         HR               4.0
6   7   Person_7  28.0   58000         IT               2.5
7   8   Person_8  32.0   72000    Finance               6.0
8   9   Person_9  45.0   90000         IT              12.0
9  10  Person_10  50.0   95000         HR              15.0


In [ ]:
# First, let's see how many missing values we have
print("\n" + "=" * 60)
print("2. Handling Missing Values")
print("معالجة القيم المفقودة")
print("=" * 60)

print("\n🔍 Missing values before cleaning:")
print("القيم المفقودة قبل التنظيف:")

# df.isnull().sum()
# - df.isnull(): Returns DataFrame with True/False (True = missing value, False = not missing)
# - .sum(): Sums True values (counts missing values) for each column
# - Returns Series with column names and count of missing values
# - Alternative: df.isna() does the same thing
missing_before = df.isnull().sum()
print(missing_before)

# missing_before.sum()
# - .sum() on Series: Adds up all values in the Series
# - This gives total missing values across all columns
print(f"\n   Total missing values: {missing_before.sum()}")

# df.shape[0] * df.shape[1]
# - df.shape[0]: Number of rows
# - df.shape[1]: Number of columns
# - shape[0] * shape[1]: Total number of cells in DataFrame
# - Used to calculate percentage of missing values
print(f"   Percentage missing: {(missing_before.sum() / (df.shape[0] * df.shape[1]) * 100):.1f}%")



2. Handling Missing Values
معالجة القيم المفقودة

🔍 Missing values before cleaning:
القيم المفقودة قبل التنظيف:
id                  0
name                0
age                 4
salary              0
department          2
experience_years    0
dtype: int64

   Total missing values: 6
   Percentage missing: 4.5%


In [ ]:
# Method 1: Remove rows with missing values
# .dropna() removes any row that has at least one missing value
# Why use this? If missing values are rare, it's better to remove than guess

print("\n--- Method 1: Remove rows with missing values ---")
print("--- الطريقة 1: حذف الصفوف ذات القيم المفقودة ---")

# df.dropna()
# - Removes rows that contain ANY missing values (NaN/None)
# - Returns new DataFrame (doesn't modify original unless inplace=True)
# - Parameters:
#   - axis=0 (default): Drop rows (axis=1 would drop columns)
#   - how='any' (default): Drop if ANY value is missing
#   - subset=None: Check all columns (can specify columns to check)
#   - inplace=False: Return new DataFrame (True modifies original)
df_removed = df.dropna()

# df.shape[0] - df_removed.shape[0]
# - df.shape[0]: Original number of rows
# - df_removed.shape[0]: Number of rows after removal
# - Difference = number of rows removed
rows_removed = df.shape[0] - df_removed.shape[0]
print(f"✅ Rows after removal: {df_removed.shape[0]} (removed {rows_removed} rows)")
print(f"الصفوف بعد الحذف: {df_removed.shape[0]} (تم حذف {rows_removed} صف)")
print(f"   Data loss: {(rows_removed / df.shape[0] * 100):.1f}%")



--- Method 1: Remove rows with missing values ---
--- الطريقة 1: حذف الصفوف ذات القيم المفقودة ---
✅ Rows after removal: 17 (removed 5 rows)
الصفوف بعد الحذف: 17 (تم حذف 5 صف)
   Data loss: 22.7%


## Step 2: Handling Missing Values | الخطوة 2: معالجة القيم المفقودة

**BEFORE**: We have missing values (NaN/None) that will break our ML models.

**AFTER**: We'll either remove rows with missing values OR fill them with reasonable estimates!

**Why handle missing values?** 
- ML algorithms cannot work with missing data
- Missing values indicate incomplete information
- We must decide: **Remove** (if few missing) or **Impute** (if many missing)

**Two main strategies:**
1. **Remove**: Drop rows/columns with missing values (good if <5% missing)
2. **Impute**: Fill missing values with mean/median/mode (good if >5% missing)

In [ ]:
# Method 2: Fill missing values (imputation)
# We'll use a copy so we don't modify the original
print("\n--- Method 2: Fill missing values ---")
print("--- الطريقة 2: ملء القيم المفقودة ---")

# df.copy()
# - Creates a deep copy of DataFrame (independent copy, not a reference)
# - Changes to copy don't affect original
# - Important: Without copy(), both variables point to same data
# - Alternative: df.copy(deep=True) is explicit (deep=True is default)
df_filled = df.copy()



--- Method 2: Fill missing values ---
--- الطريقة 2: ملء القيم المفقودة ---


In [ ]:
# Fill numeric columns with mean
# ملء الأعمدة الرقمية بالمتوسط

# df_filled['age'].fillna(df_filled['age'].mean(), inplace=True)
# - df_filled['age']: Selects 'age' column (returns Series)
# - .fillna(): Fills missing values (NaN) with specified value
#   - Parameter: Value to fill (here: mean of age column)
#   - inplace=True: Modifies DataFrame directly (False returns new Series)
# - df_filled['age'].mean(): Calculates mean (average) of age column
#   - .mean(): Returns average of all non-missing values
#   - Ignores NaN values automatically
# Result: All missing ages replaced with average age
df_filled['age'].fillna(df_filled['age'].mean(), inplace=True)

In [ ]:
# Check for and remove duplicate rows
# .duplicated() finds rows that are exact duplicates
# .drop_duplicates() removes them, keeping the first occurrence

print("\n" + "=" * 60)
print("3. Removing Duplicates")
print("إزالة التكرارات")
print("=" * 60)

# df_filled.duplicated().sum()
# - df_filled.duplicated(): Returns boolean Series (True = duplicate row, False = unique)
#   - Checks if each row is identical to a previous row
#   - First occurrence marked as False, duplicates as True
#   - Parameters:
#     - subset=None: Check all columns (can specify columns to check)
#     - keep='first' (default): Mark first as False, rest as True
# - .sum(): Counts True values (number of duplicate rows)
num_duplicates = df_filled.duplicated().sum()
print(f"\n🔍 Number of duplicates: {num_duplicates}")
print(f"عدد التكرارات: {num_duplicates}")

# df_filled.drop_duplicates()
# - Removes duplicate rows, keeps first occurrence
# - Returns new DataFrame (doesn't modify original unless inplace=True)
# - Parameters:
#   - subset=None: Check all columns for duplicates
#   - keep='first' (default): Keep first, remove rest ('last' keeps last, False removes all)
#   - inplace=False: Return new DataFrame
df_no_duplicates = df_filled.drop_duplicates()
print(f"\n✅ Rows after removing duplicates: {df_no_duplicates.shape[0]}")
print(f"الصفوف بعد إزالة التكرارات: {df_no_duplicates.shape[0]}")
print(f"   Removed {num_duplicates} duplicate row(s)")



3. Removing Duplicates
إزالة التكرارات

🔍 Number of duplicates: 2
عدد التكرارات: 2

✅ Rows after removing duplicates: 20
الصفوف بعد إزالة التكرارات: 20
   Removed 2 duplicate row(s)


In [ ]:
# Fill categorical columns with mode
# ملء الأعمدة الفئوية بالقيمة الأكثر تكراراً

# df_filled['department'].fillna(df_filled['department'].mode()[0], inplace=True)
# - df_filled['department']: Selects 'department' column
# - .mode(): Returns Series with most frequent value(s)
#   - Mode = most common value (for categorical data)
#   - Returns Series (can have multiple modes if tie)
# - [0]: Gets first mode value (if multiple modes, takes first)
# - .fillna(): Fills missing values with mode
# - inplace=True: Modifies DataFrame directly
# Result: All missing departments replaced with most common department
df_filled['department'].fillna(df_filled['department'].mode()[0], inplace=True)
print("\nMissing values after filling:")
print("القيم المفقودة بعد الملء:")
print(df_filled.isnull().sum())


Missing values after filling:
القيم المفقودة بعد الملء:
id                  0
name                0
age                 0
salary              0
department          0
experience_years    0
dtype: int64


In [ ]:
# IQR (Interquartile Range) Method for outlier detection
# This is a statistical method that identifies values far from the median

print("\n--- IQR Method for Outlier Detection ---")
print("--- طريقة المدى الربيعي للكشف عن القيم الشاذة ---")

def detect_outliers_iqr(series):
    """
    Detect outliers using IQR method.
    Returns True for outliers, False for normal values.
    """
    # series.quantile(0.25)
    # - Calculates 25th percentile (Q1) - value below which 25% of data falls
    # - quantile(q): Returns value at quantile q (0.0 to 1.0)
    # - 0.25 = 25th percentile, 0.5 = median, 0.75 = 75th percentile
    Q1 = series.quantile(0.25)  # 25th percentile
    
    # series.quantile(0.75)
    # - Calculates 75th percentile (Q3) - value below which 75% of data falls
    Q3 = series.quantile(0.75)  # 75th percentile
    
    # IQR = Q3 - Q1
    # - Interquartile Range: Spread of middle 50% of data
    # - Measures variability, less sensitive to outliers than range
    IQR = Q3 - Q1  # Interquartile Range
    
    # Q1 - 1.5 * IQR
    # - Lower fence: Values below this are considered outliers
    # - 1.5 is standard multiplier (can be adjusted)
    lower_bound = Q1 - 1.5 * IQR  # Lower fence
    
    # Q3 + 1.5 * IQR
    # - Upper fence: Values above this are considered outliers
    upper_bound = Q3 + 1.5 * IQR  # Upper fence
    
    # (series < lower_bound) | (series > upper_bound)
    # - Boolean indexing: Creates boolean Series (True = outlier, False = normal)
    # - | : Logical OR operator (element-wise)
    # - Returns True for values outside [lower_bound, upper_bound]
    # Values outside the fences are outliers
    return (series < lower_bound) | (series > upper_bound)

print("   ✅ IQR function defined")
print("   This will identify values that are too far from the median")



--- IQR Method for Outlier Detection ---
--- طريقة المدى الربيعي للكشف عن القيم الشاذة ---
   ✅ IQR function defined
   This will identify values that are too far from the median


In [12]:
# 4. Handling Outliers
# معالجة القيم الشاذة
print("\n" + "=" * 60)
print("4. Handling Outliers")
print("معالجة القيم الشاذة")
print("=" * 60)


4. Handling Outliers
معالجة القيم الشاذة


In [ ]:
# Check for outliers in salary
print("\nOutliers in salary column:")
print("القيم الشاذة في عمود الراتب:")

# detect_outliers_iqr(df_no_duplicates['salary'])
# - Calls function defined earlier
# - df_no_duplicates['salary']: Passes salary column as Series
# - Returns boolean Series (True = outlier, False = normal)
salary_outliers = detect_outliers_iqr(df_no_duplicates['salary'])

# salary_outliers.sum()
# - Counts True values (number of outliers)
print(f"Number of outliers: {salary_outliers.sum()}")

# df_no_duplicates[salary_outliers][['name', 'salary']]
# - df_no_duplicates[salary_outliers]: Boolean indexing - selects rows where salary_outliers is True
#   - Boolean Series used as filter: True rows kept, False rows removed
# - [['name', 'salary']]: Selects only 'name' and 'salary' columns
#   - Double brackets [[]] returns DataFrame (single [] returns Series)
# Result: Shows only outlier rows with name and salary columns
print(df_no_duplicates[salary_outliers][['name', 'salary']])


Outliers in salary column:
القيم الشاذة في عمود الراتب:
Number of outliers: 1
         name  salary
18  Person_19  500000


In [ ]:
# Remove outliers

# df_no_duplicates[~salary_outliers].copy()
# - ~salary_outliers: NOT operator (~) inverts boolean Series
#   - True becomes False, False becomes True
#   - Selects rows that are NOT outliers (keeps normal values)
# - [~salary_outliers]: Boolean indexing - filters DataFrame
# - .copy(): Creates independent copy (good practice)
# Result: DataFrame with outliers removed
df_clean = df_no_duplicates[~salary_outliers].copy()

In [ ]:
# Also remove impossible age values

# df_clean[df_clean['age'] <= 100].copy()
# - df_clean['age'] <= 100: Creates boolean Series (True if age <= 100, False otherwise)
#   - Comparison operator (<=) applied element-wise to all values
# - df_clean[boolean_series]: Boolean indexing - keeps rows where condition is True
# - .copy(): Creates independent copy
# Result: Keeps only rows where age is 100 or less (removes impossible ages like 150)
df_clean = df_clean[df_clean['age'] <= 100].copy()
print(f"\nRows after removing outliers: {df_clean.shape[0]}")
print(f"الصفوف بعد إزالة القيم الشاذة: {df_clean.shape[0]}")


Rows after removing outliers: 18
الصفوف بعد إزالة القيم الشاذة: 18


In [ ]:
# 5. Data Type Conversion
# تحويل أنواع البيانات
print("\n" + "=" * 60)
print("5. Data Type Conversion")
print("تحويل أنواع البيانات")
print("=" * 60)
print("\nData types before conversion:")
print("أنواع البيانات قبل التحويل:")

# df_clean.dtypes
# - Returns Series showing data type of each column
# - Common types: int64, float64, object (string), bool, datetime64
# - Useful for checking if types are correct (e.g., numbers stored as strings)
print(df_clean.dtypes)


5. Data Type Conversion
تحويل أنواع البيانات

Data types before conversion:
أنواع البيانات قبل التحويل:
id                    int64
name                 object
age                 float64
salary                int64
department           object
experience_years    float64
dtype: object


In [ ]:
# Convert experience_years to int (rounding)

# df_clean['experience_years'].round().astype(int)
# - df_clean['experience_years']: Selects 'experience_years' column
# - .round(): Rounds decimal values to nearest integer
#   - No parameter = rounds to 0 decimal places
#   - .round(2) would round to 2 decimal places
# - .astype(int): Converts data type to integer
#   - astype(): Changes data type of Series
#   - int: Integer type (int64 in pandas)
#   - Alternative: .astype('int64') or .astype(float) for float
# Result: Converts float values like 2.5 to integer 3 (after rounding)
df_clean['experience_years'] = df_clean['experience_years'].round().astype(int)

In [18]:
# Final summary of cleaning process
print("\n" + "=" * 60)
print("6. Cleaning Summary")
print("ملخص التنظيف")
print("=" * 60)

original_rows = df.shape[0]
final_rows = df_clean.shape[0]
rows_removed = original_rows - final_rows

print(f"\n📊 Original rows: {original_rows}")
print(f"الصفوف الأصلية: {original_rows}")
print(f"✅ Final cleaned rows: {final_rows}")
print(f"الصفوف النهائية بعد التنظيف: {final_rows}")
print(f"\n🗑️  Rows removed: {rows_removed} ({(rows_removed/original_rows*100):.1f}%)")
print(f"الصفوف المحذوفة: {rows_removed}")

print("\n📄 Cleaned Data (first 10 rows):")
print("البيانات النظيفة:")
print(df_clean.head(10))

print("\n" + "=" * 60)
print("✅ Example 2 Complete! ✓")
print("اكتمل المثال 2! ✓")
print("=" * 60)
print("\n🎓 What you accomplished:")
print("   ✅ Handled missing values (imputed with mean/mode)")
print("   ✅ Removed duplicate rows")
print("   ✅ Detected and removed outliers")
print("   ✅ Converted data types correctly")
print("   ✅ Created clean, model-ready data!")



6. Cleaning Summary
ملخص التنظيف

📊 Original rows: 22
الصفوف الأصلية: 22
✅ Final cleaned rows: 18
الصفوف النهائية بعد التنظيف: 18

🗑️  Rows removed: 4 (18.2%)
الصفوف المحذوفة: 4

📄 Cleaned Data (first 10 rows):
البيانات النظيفة:
   id       name   age  salary department  experience_years
0   1   Person_1  25.0   50000         IT                 2
1   2   Person_2  30.0   60000         HR                 5
2   3   Person_3  40.5   55000         IT                 3
3   4   Person_4  35.0   70000    Finance                 8
4   5   Person_5  40.0   80000         IT                10
5   6   Person_6  40.5   65000         HR                 4
6   7   Person_7  28.0   58000         IT                 2
7   8   Person_8  32.0   72000    Finance                 6
8   9   Person_9  45.0   90000         IT                12
9  10  Person_10  50.0   95000         HR                15

✅ Example 2 Complete! ✓
اكتمل المثال 2! ✓

🎓 What you accomplished:
   ✅ Handled missing values (imputed with

## Step 6: Decision Framework - When to Remove vs. When to Fix | الخطوة 6: إطار القرار - متى نحذف ومتى نصلح

**BEFORE**: You've learned different cleaning methods, but when should you use each one?

**AFTER**: You'll have a clear decision framework to choose the right cleaning method for any situation!

**Why this matters**: Making the wrong cleaning decision can:
- **Remove too much data** → Lose valuable information
- **Keep bad data** → Break your models
- **Use wrong method** → Introduce bias or errors

---

### 🎯 Decision Framework for Missing Values | إطار القرار للقيم المفقودة

**Key Question**: Should I **REMOVE** or **IMPUTE** missing values?

#### Decision Tree:

```
Is missing data < 5% of total?
├─ YES → REMOVE (dropna)
│   └─ Why? Small loss, keeps data "pure"
│
└─ NO → Is missing data random or systematic?
    ├─ RANDOM → IMPUTE (fillna with mean/median/mode)
    │   └─ Why? Random missing = no bias, safe to estimate
    │
    └─ SYSTEMATIC → INVESTIGATE FIRST
        └─ Why? Systematic missing might indicate important pattern
```

#### Comparison Table:

| Method | When to Use | Pros | Cons | Example |
|--------|-------------|------|------|---------|
| **Remove** | < 5% missing, random | • No bias introduced<br>• Keeps data "pure"<br>• Simple | • Loses data<br>• Can't use if many missing | Age missing in 2% of records |
| **Impute (Mean/Median)** | > 5% missing, numeric, random | • Keeps all rows<br>• Preserves sample size<br>• Works for numeric | • Can introduce bias<br>• Assumes normal distribution | Salary missing in 15% of records |
| **Impute (Mode)** | > 5% missing, categorical, random | • Keeps all rows<br>• Preserves sample size<br>• Works for categories | • Can create artificial patterns<br>• May over-represent common values | Department missing in 10% of records |
| **Investigate** | Systematic missing | • Finds root cause<br>• Prevents bias<br>• Better decisions | • Takes time<br>• Requires domain knowledge | All salaries missing for one department |

---

### 🎯 Decision Framework for Outliers | إطار القرار للقيم الشاذة

**Key Question**: Should I **REMOVE** or **KEEP** outliers?

#### Decision Tree:

```
Is the outlier a data entry error?
├─ YES → REMOVE
│   └─ Example: Age = 150, Salary = 500000 (typo)
│
└─ NO → Is the outlier a rare but valid event?
    ├─ YES → KEEP (but handle separately)
    │   └─ Example: CEO salary in employee dataset
    │
    └─ NO → Does it affect model performance?
        ├─ YES → REMOVE or TRANSFORM
        │   └─ Example: Extreme values breaking linear regression
        │
        └─ NO → KEEP
            └─ Example: Outlier in non-critical feature
```

#### Comparison Table:

| Method | When to Use | Pros | Cons | Example |
|--------|-------------|------|------|---------|
| **Remove** | Data entry errors, impossible values | • Removes noise<br>• Improves model accuracy<br>• Simple | • Loses information<br>• May remove valid rare events | Age = 150, Salary = 500000 |
| **Cap/Clip** | Valid but extreme values | • Keeps data<br>• Reduces impact<br>• Preserves distribution | • Arbitrary threshold<br>• May hide important patterns | Cap salary at 99th percentile |
| **Transform** | Skewed distributions | • Normalizes data<br>• Keeps all values<br>• Better for ML | • Changes interpretation<br>• More complex | Log transform for income |
| **Keep** | Rare but valid events | • Preserves reality<br>• No information loss<br>• Important for analysis | • Can skew models<br>• May need special handling | CEO in employee dataset |

---

### 🎯 Decision Framework for Duplicates | إطار القرار للتكرارات

**Key Question**: Should I **REMOVE** all duplicates or **INVESTIGATE** first?

#### Decision Tree:

```
Are duplicates exact copies?
├─ YES → REMOVE (keep first)
│   └─ Why? No new information, wastes space
│
└─ NO → Are duplicates near-duplicates (typos)?
    ├─ YES → FIX (merge or correct)
    │   └─ Why? Same entity, different spelling
    │
    └─ NO → Are duplicates valid (same person, different records)?
        └─ YES → KEEP (but flag for analysis)
            └─ Why? Important information (e.g., repeat customers)
```

---

### 📊 Real-World Examples | أمثلة من العالم الحقيقي

#### Example 1: E-commerce Dataset
- **Missing values in "price"**: 20% missing
  - **Decision**: IMPUTE with median (too much to remove, random missing)
  - **Reason**: Random missing prices, median preserves distribution

#### Example 2: Medical Dataset
- **Outlier in "age"**: One patient age = 200
  - **Decision**: REMOVE (impossible value)
  - **Reason**: Data entry error, no one lives to 200

#### Example 3: Customer Dataset
- **Missing values in "email"**: 3% missing
  - **Decision**: REMOVE (small percentage)
  - **Reason**: Email is critical, can't impute, small loss acceptable

#### Example 4: Sales Dataset
- **Outlier in "revenue"**: One sale = $1,000,000 (normal range: $10-$1000)
  - **Decision**: INVESTIGATE FIRST
  - **Reason**: Could be valid (enterprise sale) or error (extra zero)

---

### ✅ Key Takeaways | النقاط الرئيسية

1. **Always investigate first** - Understand WHY data is missing/outlier/duplicate
2. **Consider data loss** - Removing >10% of data is usually too much
3. **Consider bias** - Systematic missing/outliers may indicate important patterns
4. **Test both approaches** - Sometimes try removing AND imputing, compare results
5. **Document decisions** - Write down WHY you chose each method (for reproducibility)

---

### 🎓 Practice Decision-Making | ممارسة اتخاذ القرار

**Scenario**: You have a dataset with:
- 8% missing values in "income" column
- 2 outliers in "age" (ages 0 and 200)
- 5 duplicate rows

**Your task**: Decide what to do for each issue and explain why!

**Answers**:
1. **Income (8% missing)**: IMPUTE with median (too much to remove, likely random)
2. **Age outliers**: REMOVE (impossible values - data entry errors)
3. **Duplicates**: REMOVE (exact copies, no new information)


In [19]:
# Practical Example: Comparing Different Cleaning Approaches
# مثال عملي: مقارنة طرق التنظيف المختلفة

print("=" * 60)
print("Practical Example: Decision-Making in Action")
print("مثال عملي: اتخاذ القرار في الممارسة")
print("=" * 60)

# Create a scenario with different data quality issues
print("\n📊 Scenario: Dataset with multiple issues")
print("   - 15% missing values in 'income' (too much to remove)")
print("   - 2% missing values in 'email' (can remove)")
print("   - 1 outlier: age = 200 (impossible, should remove)")
print("   - 3 duplicates (should remove)")

# Simulate the decision-making process
print("\n" + "-" * 60)
print("DECISION 1: Income missing (15%)")
print("-" * 60)
print("   ❌ Can't remove: Would lose 15% of data (too much!)")
print("   ✅ Should impute: Use median (preserves distribution)")
print("   📝 Reason: Random missing, large percentage, numeric data")

print("\n" + "-" * 60)
print("DECISION 2: Email missing (2%)")
print("-" * 60)
print("   ✅ Can remove: Only 2% loss (acceptable)")
print("   ❌ Can't impute: Email is unique, can't estimate")
print("   📝 Reason: Small percentage, critical field, can't fill")

print("\n" + "-" * 60)
print("DECISION 3: Age outlier (age = 200)")
print("-" * 60)
print("   ✅ Should remove: Impossible value (data entry error)")
print("   ❌ Can't keep: Would break age-based models")
print("   📝 Reason: No human lives to 200, clearly an error")

print("\n" + "-" * 60)
print("DECISION 4: Duplicates (3 rows)")
print("-" * 60)
print("   ✅ Should remove: Exact duplicates, no new information")
print("   📝 Reason: Wastes space, can bias models (same data counted twice)")

print("\n" + "=" * 60)
print("✅ Decision Framework Applied Successfully!")
print("تم تطبيق إطار القرار بنجاح!")
print("=" * 60)


Practical Example: Decision-Making in Action
مثال عملي: اتخاذ القرار في الممارسة

📊 Scenario: Dataset with multiple issues
   - 15% missing values in 'income' (too much to remove)
   - 2% missing values in 'email' (can remove)
   - 1 outlier: age = 200 (impossible, should remove)
   - 3 duplicates (should remove)

------------------------------------------------------------
DECISION 1: Income missing (15%)
------------------------------------------------------------
   ❌ Can't remove: Would lose 15% of data (too much!)
   ✅ Should impute: Use median (preserves distribution)
   📝 Reason: Random missing, large percentage, numeric data

------------------------------------------------------------
DECISION 2: Email missing (2%)
------------------------------------------------------------
   ✅ Can remove: Only 2% loss (acceptable)
   ❌ Can't impute: Email is unique, can't estimate
   📝 Reason: Small percentage, critical field, can't fill

----------------------------------------------------

In [ ]:
# Convert age to int

# df_clean['age'].round().astype(int)
# - df_clean['age']: Selects 'age' column
# - .round(): Rounds decimal values (e.g., 40.5 → 41)
# - .astype(int): Converts to integer type
# Result: Converts float ages to integer ages
df_clean['age'] = df_clean['age'].round().astype(int)
print("\nData types after conversion:")
print("أنواع البيانات بعد التحويل:")

# df_clean.dtypes
# - Shows data types after conversion
# - Should show int64 for age and experience_years now
print(df_clean.dtypes)


Data types after conversion:
أنواع البيانات بعد التحويل:
id                   int64
name                object
age                  int64
salary               int64
department          object
experience_years     int64
dtype: object
